In [51]:
from time import time
import pandas as pd
import os
import sys
sys.path.insert(0, os.path.abspath('../graphs'))


from graphs import MatchingGraph, GraphCsv, GraphMysql

In [2]:
seed_1 = {'source': 600971, 'e_type': 4, 'target': 579269, 'time': -685755382, 'weight': 0.166667, 'source_location': 0, 'target_location': 0, 'source_latitude': 0, 'source_longitude': 0, 'target_latitude': 0, 'target_longitude': 0}
seed_2 = {'source': 538771, 'e_type': 4, 'target': 473043, 'time': -623491200, 'weight': 0.0909091, 'source_location': 0, 'target_location': 0, 'source_latitude': 0, 'source_longitude': 0, 'target_latitude': 0, 'target_longitude': 0}
seed_3 = {'source': 574136, 'e_type': 2, 'target': 657187, 'time': 1991785, 'weight': 633, 'source_location': 0, 'target_location': 0, 'source_latitude': 0, 'source_longitude': 0, 'target_latitude': 0, 'target_longitude': 0}
mysql_config = {'hostname': 'localhost', 'user': 'root', 'password': 'root', 'schema': 'vast', 'port': 52000}
large_graph = GraphMysql(mysql_config)
template_graph = GraphCsv('./template_nodes.csv', './template_links.csv')
matching_graph = MatchingGraph(large_graph, template_graph)

In [3]:
print(seed_1)
print(seed_2)
print(seed_3)
hub_nodes = [39, 40, 41]

{'source': 600971, 'e_type': 4, 'target': 579269, 'time': -685755382, 'weight': 0.166667, 'source_location': 0, 'target_location': 0, 'source_latitude': 0, 'source_longitude': 0, 'target_latitude': 0, 'target_longitude': 0}
{'source': 538771, 'e_type': 4, 'target': 473043, 'time': -623491200, 'weight': 0.0909091, 'source_location': 0, 'target_location': 0, 'source_latitude': 0, 'source_longitude': 0, 'target_latitude': 0, 'target_longitude': 0}
{'source': 574136, 'e_type': 2, 'target': 657187, 'time': 1991785, 'weight': 633, 'source_location': 0, 'target_location': 0, 'source_latitude': 0, 'source_longitude': 0, 'target_latitude': 0, 'target_longitude': 0}


In [5]:
t0 = time()
mirror_nodes_seed1 = matching_graph.walk_graph(600971, 0, hub_nodes)
print(mirror_nodes_seed1)
print(time() - t0)

Finding similarity in communications
Calculando similares de (original) 600971 y (template) 0
Empty links
calculate_similar_nodes
Similares como source: {}
calculate_similar_nodes
Similares como target: {40: (457543, 1.8522534708937524), 41: (611974, 1.9111347439532855), 65: (488050, 0.9633098420365317)}
Calculando similares de (original) 488050 y (template) 65
calculate_similar_nodes
Similares como source: {39: (606702, 1.707829650287816)}
calculate_similar_nodes
Similares como target: {}
Calculando similares de (original) 606702 y (template) 39
calculate_similar_nodes
Similares como source: {66: (523763, 0), 47: (620279, 0.9609062949481668), 67: (528228, 0)}
calculate_similar_nodes
Similares como target: {67: (645586, 0), 47: (458232, 0.9696471935351735)}
Calculando similares de (original) 523763 y (template) 66
Empty links
calculate_similar_nodes
Similares como source: {}
calculate_similar_nodes
Similares como target: {}
Calculando similares de (original) 458232 y (template) 47
calc

In [4]:
t0 = time()
mirror_nodes_seed2 = matching_graph.walk_graph(538771, 0, hub_nodes)
print(mirror_nodes_seed2)
print(time() - t0)

Finding similarity in communications
Calculando similares de (original) 538771 y (template) 0
Empty links
Empty links
calculate_similar_nodes
Similares como source: {}
Empty links
calculate_similar_nodes
Similares como target: {40: (-1, -1), 41: (-1, -1), 65: (-1, -1)}
Calculando similares de (original) -1 y (template) 65
Empty links
calculate_similar_nodes
Similares como source: {39: (-1, -1)}
Empty links
calculate_similar_nodes
Similares como target: {}
Calculando similares de (original) -1 y (template) 39
Empty links
calculate_similar_nodes
Similares como source: {66: (-1, -1), 47: (-1, -1), 67: (-1, -1)}
Empty links
calculate_similar_nodes
Similares como target: {67: (-1, -1), 47: (-1, -1)}
Calculando similares de (original) -1 y (template) 66
Empty links
Empty links
calculate_similar_nodes
Similares como source: {}
Empty links
calculate_similar_nodes
Similares como target: {}
Calculando similares de (original) -1 y (template) 47
Empty links
calculate_similar_nodes
Similares como s

In [6]:
t0 = time()
mirror_nodes_seed3 = matching_graph.walk_graph(574136, 67, hub_nodes)
print(mirror_nodes_seed3)
print(time() - t0)


Finding similarity in communications
Calculando similares de (original) 574136 y (template) 67
calculate_similar_nodes
Similares como source: {39: (499382, 1.7569431858897164), 47: (490458, 0.9538832494921798)}
calculate_similar_nodes
Similares como target: {39: (461437, 1.7576196705323368), 47: (470528, 0.967653311613888)}
Calculando similares de (original) 470528 y (template) 47
calculate_similar_nodes
Similares como source: {27: (551765, 0.9887216630457052), 57: (585654, 0.923416793221785), 45: (614439, 0.98861402971856), 58: (552044, 0.9690908723821873), 56: (471475, 0.9739561021535967), 63: (617353, 0.9952599218476157), 41: (649553, 1.910174446367406), 43: (527537, 0.9839375750544733)}
calculate_similar_nodes
Similares como target: {56: (482097, 0.9735206614133969), 27: (488367, 0.9870285837275422), 41: (611974, 1.9111347439532855), 63: (474512, 0.9948134614880384), 58: (604641, 0.9652228047581526), 37: (648109, 0.9884006433749246), 45: (594154, 0.9860009108216452), 43: (565605, 0

In [63]:
matching_communications_seed_1 = {0: 600971,40: 457543,41: 611974,65: 488050,39: 606702,
                                  67: 645586,47: 458232,66: 523763,43: 494819,45: 594766,
                                  56: 610377,58: 652117,27: 533094,63: 541601,57: 491440,
                                  37: 564599,34: 598958}

matching_travels_seed_1 = {74: 634234,75: 477668,73: 636858,87: 510261,85: 506949,
                           86: 487516,80: 625689,77: 494508,79: 629627,78: 599441,
                           82: 533956,83: 584896,84: 463206,
                           40: 457543,41: 611974,39: 606702}
matching_values_communications_1 = list(matching_communications_seed_1.values())
matching_values_travels_1 = list(matching_travels_seed_1.values())

In [33]:
matching_communications_seed_3 = {67: 574136,39: 461437,47: 470528,41: 611974,43: 527537,
                                  45: 614439,56: 471475,58: 552044,27: 551765,63: 617353,
                                  57: 585654,37: 648109,34: 522487,40: 482596,65: 593045,
                                  0: 506586,66: 533691}

matching_travels_seed_3 = {80: 625689,77: 494508,79: 629627,78: 599441,73: 470622,
                           75: 634234,74: 477668,86: 583014,87: 510261,85: 471904,
                           82: 533956,83: 584896,84: 463206,
                           39: 461437,40: 482596,41: 611974}
matching_values_communications_3 = list(matching_communications_seed_3.values())
matching_values_travels_3 = list(matching_travels_seed_3.values())

In [84]:
def build_csv_seed_files(graph, communication_nodes, travel_nodes):
    links = pd.DataFrame()
    # Reconstruyo con las relaciones de comunicaciones y financieras
    for a_node in communication_nodes:
        sub_select = graph.find_by_params(target=a_node, e_types=[0, 1])
        temp_data_frame = sub_select[sub_select['source'].isin(communication_nodes)]
        links = links.append(temp_data_frame, ignore_index=True)

        sub_select = graph.find_by_params(source=a_node, e_types=[0, 1])
        temp_data_frame = sub_select[sub_select['target'].isin(communication_nodes)]
        links = links.append(temp_data_frame, ignore_index=True)

        sub_select = graph.find_by_params(target=a_node, e_types=[5])
        links = links.append(sub_select, ignore_index=True)

        sub_select = graph.find_by_params(source=a_node, e_types=[5])
        links = links.append(sub_select, ignore_index=True)

    for a_node in travel_nodes:
        sub_select = graph.find_by_params(source=a_node, e_types=[6])
        links = links.append(sub_select, ignore_index=True)

    nodes_ids = list(set(links['source']) | set(links['target']))
    nodes_types = []
    for a_node in nodes_ids:
        nodes_types.append(graph.find_node_type(a_node))

    nodes = pd.DataFrame({'id': nodes_ids, 'node_type': nodes_types})
    return links, nodes

In [87]:
links_1, nodes_1 =build_csv_seed_files(large_graph, matching_values_communications_1, matching_values_travels_1)
links_3, nodes_3 =build_csv_seed_files(large_graph, matching_values_communications_3, matching_values_travels_3)

In [99]:
links_1.columns = ['Source', 'eType', 'Target', 'Time', 'Weight', 'SourceLocation',
       'TargetLocation', 'SourceLatitude', 'SourceLongitude', 'TargetLatitude',
       'TargetLongitude']
nodes_1.columns = ['NodeID', 'NodeType']

links_3.columns = ['Source', 'eType', 'Target', 'Time', 'Weight', 'SourceLocation',
       'TargetLocation', 'SourceLatitude', 'SourceLongitude', 'TargetLatitude',
       'TargetLongitude']
nodes_3.columns = ['NodeID', 'NodeType']

In [100]:
links_1.to_csv("./links_seed_1.csv", header=True, index=False)
nodes_1.to_csv("./nodes_seed_1.csv", header=True, index=False)
links_3.to_csv("./links_seed_3.csv", header=True, index=False)
nodes_3.to_csv("./nodes_seed_3.csv", header=True, index=False)

Agrego las columnas que faltan para el notebook
SourceNodeType, TargetNodeType, SourceNodeDescription, TargetNodeDescription, EdgeDescription

In [104]:
node_types = pd.read_csv('./data/NodeTypeDescriptions.csv')
edge_types = pd.read_csv('./data/EdgeTypesDescription.csv')
financial_types = pd.read_csv('./data/DemographicCategories.csv')
financial_types

,NodeID,Category
0,459381,Water and other public services
1,466907,Electricity
2,473173,Household furnishings
3,503218,Natural gas
4,503701,Miscellaneous
5,510031,Gifts
6,520660,Healthcare
7,523927,Restaurants
8,527449,Alcohol
9,536346,Home maintenance
